In [1]:
# Load .env file 
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())

# Create LLM instance
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL")
)

1. example: PydanticOutputParser

In [2]:
# Schema for structured output
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class SearchQuery(BaseModel):
    search_query: str = Field(
        None,
        description="Query that is optimized web search."
    )
    
    justification: str = Field(
        None, description="Why this query is relevant to the user's request."
    )
    
    naive_answer: str = Field(
        None, description="Answer user query."
    )
    
    
# Create OutputParser
parser = PydanticOutputParser(
    pydantic_object=SearchQuery
)    

In [3]:
# Create PromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert at converting user questions into optimized search queries.
Given a question or request, generate an optimized search query and explain why it's relevant.
    {format_instructions}"""),
    ("human", "{question}")
])

In [4]:
# create query analyzer chain 
query_analyzer = prompt.partial(format_instructions=parser.get_format_instructions()) | llm | parser 

# 
result = query_analyzer.invoke({"question": "How does Calcium CT score relate to high cholesterol?"})

In [5]:
result

SearchQuery(search_query='Calcium CT score relation to high cholesterol cardiovascular risk', justification='The search query is optimized to focus on the relationship between Calcium CT score (also known as coronary artery calcium score) and high cholesterol, particularly in the context of cardiovascular risk. These terms are commonly used in medical literature and web resources on this topic.', naive_answer='The Calcium CT score is used to assess the amount of calcium in coronary arteries, which can indicate the presence of atherosclerosis. High cholesterol can contribute to the buildup of plaque in these arteries, leading to a higher Calcium CT score. This correlation helps in estimating cardiovascular risk.')

In [6]:
display(parser.get_format_instructions())

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"search_query": {"default": null, "description": "Query that is optimized web search.", "title": "Search Query", "type": "string"}, "justification": {"default": null, "description": "Why this query is relevant to the user\'s request.", "title": "Justification", "type": "string"}, "naive_answer": {"default": null, "description": "Answer user query.", "title": "Naive Answer", "type": "string"}}}\n```'

2. example : get_weather() with LangChain

In [7]:
# create get_weather func with pydantic
import requests
from typing import Optional 

class WeatherResponse(BaseModel):
    temperature: float = Field(..., description="Current temperature in Celsius")
    temperature_feellike: float = Field(..., description="Feellike temperature in Celsius"),
    temperature_min: float = Field(..., description="Maximum temperature in day in Celsius"),
    temperature_max: float = Field(..., description="Minimum temperature in day in Celsius"),
    humidity: int = Field(..., description="Current humidity percentage")
    description: str = Field(..., description="Weather description")
    city: str = Field(..., description="City name in english")


def get_weather(location: str) -> WeatherResponse:
    """Get the current weather for a given location"""
    weather_api_key = os.getenv("OPENWEATHER_API_KEY")
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'appid': weather_api_key,
        'q': location,
        'units': 'metric',
        #'lang': 'zh_cn'
    }
    try:
        response = requests.get(base_url, params)
        response.raise_for_status()
        data = response.json()
        return WeatherResponse(
            temperature=data['main']['temp'],
            temperature_feellike=data['main']['feels_like'],
            temperature_min=data['main']['temp_min'],
            temperature_max=data['main']['temp_max'],
            humidity=data['main']['humidity'],
            description=data['weather'][0]['description'],
            city=data['name']
        )

    except requests.RequestException as e:
        raise Exception(f"Error fetch weather data from function get_weather(): {str(e)}")
    
# test
get_weather("Hangzhou")

WeatherResponse(temperature=10.95, temperature_feellike=9.64, temperature_min=10.95, temperature_max=10.95, humidity=59, description='overcast clouds', city='Hangzhou')

In [8]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.tools import StructuredTool
from langchain.agents import AgentExecutor, create_openai_tools_agent

# create weather tools 
weather_tool = StructuredTool.from_function(
    func=get_weather,
    name="get_weather",
    description="Get the current weather for given location. Input should be a city name in English."
)

# create chat template for get_weather
# template = """You are a helpful assistant that provides weather information.
# Use the provided tools to answer questions about the weather.

# Question: {question}

# {agent_scratchpad}"""


prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant. You will response user's question in nature and consistence way."""),
    ("user", "{question}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# create agent 
agent = create_openai_tools_agent(llm, [weather_tool], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[weather_tool])

In [10]:
# 使用示例 # What's the weather like in Hangzhou right now?
result = agent_executor.invoke({
    "question": "你能告诉哦我现在杭州的天气吗？"
})
print(result["output"])

result = agent_executor.invoke({"question": "物理学中的基态是什么意思？"})
print(result["output"])

Exception: Error fetch weather data from function get_weather(): 404 Client Error: Not Found for url: http://api.openweathermap.org/data/2.5/weather?appid=2d2b36fc156aabe2b73768a896af1df6&q=%E6%9D%AD%E5%B7%9E&units=metric

3. example: numeric operations with LangGraph

In [ ]:
# create simple numeric operations with agent.tools 
from langchain_core import tool 

@tool
def add(a: float, b: float) -> float:
    """Add a and b in float type
    Args:
        a (float): first float number 
        b (float): second float number
    Returns:
        float: result of a + b
    """
    return a + b 

@tool
def divide(a: float, b: float) -> float:
    """Divide a and b in float type

    Args:
        a (float): first float number
        b (float): second float number 

    Returns:
        float: result of a - b 
    """
    return a - b  

@tool 
def multiply(a: float, b:float) -> float:
    """Multiply a and b 
    Args:
        a (float): first float number
        b (float): first float number
    Returns: 
        float: result of a * b 
    """
    return a * b 

@tool 
def division(a: float, b: float) -> float:
    """Divison a and b 
    Args:
        a (float): first float number
        b (float): second float number

    Returns:
        float: result of a / b 
    """
    return a / b 

# create tools set 
tools = [add, divide, multiply, division]
tools_by_name = {tool.name for tool in tools}
llm_with_tools = llm.bind_tools(tools)
    

In [11]:
from langgraph.graph import StateGraph
from typing import TypedDict, List, Dict 

# Define GraphState 
class AgentState(TypedDict):
    messages: List[Dict]
    current_input: str
    result: List[float]
    
# 创建图
workflow = StateGraph(AgentState)

# 创建使用工具的节点函数
def math_operation_node(state: AgentState) -> Dict:
    # 使用 llm_with_tools 处理输入
    response = llm_with_tools.invoke(state["current_input"])
    
    # 更新状态
    return {
        "messages": state["messages"] + [{"role": "assistant", "content": str(response)}],
        "results": state["results"] + [float(response)]
    }

# 添加节点
workflow.add_node("math_solver", math_operation_node)

# 设置入口点和结束点
workflow.set_entry_point("math_solver")
workflow.set_finish_point("math_solver")

# 编译图
app = workflow.compile()

In [ ]:


# 调用图
initial_state = {
    "messages": [],
    "current_input": "请将 5 和 3 相加",
    "results": []
}

result = app.invoke(initial_state)
